In [2]:
import ollama
import os

import numpy as np


In [2]:
def embed(doc):
    return ollama.embed(model="nomic-embed-text", input=doc)['embeddings']
    #model="avr/sfr-embedding-mistral"
#vector_store.add_documents(documents=chunks, ids=uuids)

In [2]:
#use sklearn
from sklearnex import patch_sklearn
patch_sklearn()

from  sklearn.neighbors import NearestNeighbors

short_search = NearestNeighbors(n_neighbors=5, radius=1.0, algorithm='auto', metric='cosine')

long_search = NearestNeighbors(n_neighbors=5, radius=1.0, algorithm='auto', metric='cosine')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
import joblib
import h5py
import json
from tqdm import tqdm

In [4]:

long_chunks = joblib.load('chunks.pkl')

long_chunks_embed = np.zeros((len(long_chunks), 768))
long_strings =[]
for i in tqdm(range(len(long_chunks))):
    content = json.dumps(
            long_chunks[i].page_content)
    long_chunks_embed[i] = np.array(embed(content))
    long_strings.append(content)

print(long_chunks_embed[0])

#long_embed = np.vstack(long_chunks_embed)

long_search.fit(long_chunks_embed)

joblib.dump(long_search, 'long_search_sk_nomic.pkl')


100%|████████████████████████████████████████████| 1246108/1246108 [3:03:36<00:00, 113.11it/s]


[-1.98466140e-02 -4.76848600e-02 -1.39799910e-01 -7.29522000e-02
 -2.31321710e-02  2.18195620e-02  2.63258890e-02  4.33079000e-02
  1.25413750e-02  1.87532980e-02 -4.40246150e-03 -1.56123570e-02
  8.99769300e-02  6.13842940e-02 -6.78549500e-03 -6.01059940e-02
 -3.46542560e-02 -2.05939800e-02 -7.15108740e-02 -1.15704670e-02
 -1.87881380e-02 -6.47210800e-02 -1.90031940e-02 -2.49527430e-02
  6.60239600e-02  2.28594820e-02 -5.34723400e-02 -4.07897530e-02
 -7.17271200e-02 -1.88137440e-02  5.17141860e-02 -7.24393200e-02
 -6.76922500e-03  8.32715800e-03 -2.97793100e-02 -5.72318360e-02
  1.48887330e-02 -5.13244400e-02 -3.78479360e-02 -3.36908300e-02
  7.97627340e-02  1.82351880e-02 -6.52575170e-03 -3.28630250e-02
  1.71768030e-02  4.08118330e-02  5.13687730e-02 -9.65188600e-02
  6.82348100e-02 -5.52231730e-03  5.44730240e-02 -9.49503700e-03
  2.24255700e-02 -3.96561340e-03  1.80207960e-02  1.26891140e-02
 -1.28565410e-02 -2.37938240e-02  9.92143300e-02 -3.36379500e-02
  1.01818986e-01  2.95439

['long_search_sk_nomic.pkl']

In [5]:
with h5py.File("nomic_db.hdf5", "w") as f:
    long_set = f.create_dataset("long_chunks", (len(long_strings),), maxshape = (None,), dtype=h5py.string_dtype())
    long_set[:] = long_strings

In [6]:
"""
with h5py.File("chunks_db.hdf5", "r") as f:
    print(str(f['long_chunks'][12500]))
"""

'\nwith h5py.File("chunks_db.hdf5", "r") as f:\n    print(str(f[\'long_chunks\'][12500]))\n'

In [4]:

from cuml.neighbors import NearestNeighbors as cuNN
short_search = cuNN(metric = 'cosine')
"""
long_cusearch = cuNN(metric = 'cosine')
long_cusearch.fit(long_chunks_embed)
"""

"\nlong_cusearch = cuNN(metric = 'cosine')\nlong_cusearch.fit(long_chunks_embed)\n"

In [8]:
"""
l2_cusearch = cuNN(metric = 'l2')
l2_cusearch.fit(long_chunks_embed)
joblib.dump(l2_cusearch, 'cul2search.pkl')
"""

"\nl2_cusearch = cuNN(metric = 'l2')\nl2_cusearch.fit(long_chunks_embed)\njoblib.dump(l2_cusearch, 'cul2search.pkl')\n"

In [9]:
#joblib.dump(long_cusearch, 'cusearch.pkl')

In [10]:
joblib.dump(long_chunks_embed, 'long_nomic_embed.pkl')

['long_nomic_embed.pkl']

In [4]:
short_chunks = joblib.load('short_chunks.pkl')

short_chunks_embed = []

short_chunks_embed = np.zeros((len(short_chunks), 768))
short_strings =[]
for i in tqdm(range(len(short_chunks))):
    content = json.dumps(
            short_chunks[i].page_content)
    #short_chunks_embed[i] = np.array(embed(content))
    short_strings.append(content)
    







100%|███████████████████████████████████████████| 2977510/2977510 [00:03<00:00, 956565.03it/s]


In [6]:
#joblib.dump(short_chunks_embed, 'short_chunks_embed.pkl')

['short_chunks_embed.pkl']

In [5]:
#short_chunks_embed = joblib.load('short_chunks_embed.pkl')

In [6]:
from umap import UMAP

In [7]:
"""
s_umap = UMAP(n_components =50, n_jobs = -1)
from sklearn.model_selection import train_test_split

random_chunks,_ = train_test_split(short_chunks_embed, test_size = 0.95)
s_umap.fit_transform(random_chunks)
umap_embed = s_umap.transform(short_chunks_embed)
"""

In [9]:
"""
sumap_search = cuNN()
sumap_search.fit(umap_embed)

joblib.dump(sumap_search, 'sumap_search.pkl')
joblib.dump(s_umap, 's_umap.pkl')
"""

['s_umap.pkl']

In [5]:
#short_search.fit(short_chunks_embed)

#joblib.dump(short_search, 'short_cusearch_nomic.pkl')

with h5py.File("nomic_short.hdf5", "w") as f:
    short_set = f.create_dataset("short_chunks", (len(short_strings),), maxshape = (None,), dtype=h5py.string_dtype())
    short_set[:] = short_strings
